# COVID-19 Geocoder

In [1]:
# import packages
import pandas as pd
import requests
import logging
import time

In [2]:
## Geocoding:
## script credit to: https://github.com/shanealynn/python_batch_geocode/blob/master/python_batch_geocoding.py

In [3]:
logger = logging.getLogger("root")
logger.setLevel(logging.DEBUG)
# create console handler
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
logger.addHandler(ch)

# place your own "API_KEY" here to do geocoding!!! 
API_KEY = ''
BACKOFF_TIME = 30
address_column_name = "Address"
output_filename = "Vac_station_geocoded.csv"
RETURN_FULL_RESULTS = False

In [4]:
data = pd.read_csv('Vac_station_final.csv')
if address_column_name not in data.columns:
    raise ValueError("Missing Address column in input data")
    

addresses = (data[address_column_name] + ', NY').to_list()
addresses[0:5]

['132 West 138th Street, Manhattan, NY',
 '2309 Frederick Douglass Blvd, Manhattan, NY',
 '140 Flatbush Avenue, Brooklyn, NY',
 '57 Bay Street, Staten Island, NY',
 'Bay 32nd St & Falcon Ave, Far Rockaway, Queens, NY']

In [5]:
def get_google_results(address, api_key=None, return_full_response=False):
    """
    Get geocode results from Google Maps Geocoding API.
    
    Note, that in the case of multiple google geocode reuslts, this function returns details of the FIRST result.
    
    @param address: String address as accurate as possible. For Example "18 Grafton Street, Dublin, Ireland"
    @param api_key: String API key if present from google. 
                    If supplied, requests will use your allowance from the Google API. If not, you
                    will be limited to the free usage of 2500 requests per day.
    @param return_full_response: Boolean to indicate if you'd like to return the full response from google. This
                    is useful if you'd like additional location details for storage or parsing later.
    """
    # Set up your Geocoding url
    geocode_url = "https://maps.googleapis.com/maps/api/geocode/json?address={}".format(address)
    if api_key is not None:
        geocode_url = geocode_url + "&key={}".format(api_key)
        
    # Ping google for the reuslts:
    results = requests.get(geocode_url)
    # Results will be in JSON format - convert to dict using requests functionality
    results = results.json()
    
    # if there's no results or an error, return empty results.
    if len(results['results']) == 0:
        output = {
            "formatted_address" : None,
            "latitude": None,
            "longitude": None,
            "accuracy": None,
            "google_place_id": None,
            "type": None,
            "postcode": None
        }
    else:    
        answer = results['results'][0]
        output = {
            "formatted_address" : answer.get('formatted_address'),
            "latitude": answer.get('geometry').get('location').get('lat'),
            "longitude": answer.get('geometry').get('location').get('lng'),
            "accuracy": answer.get('geometry').get('location_type'),
            "google_place_id": answer.get("place_id"),
            "type": ",".join(answer.get('types')),
            "postcode": ",".join([x['long_name'] for x in answer.get('address_components') 
                                  if 'postal_code' in x.get('types')])
        }
        
    # Append some other details:    
    output['input_string'] = address
    output['number_of_results'] = len(results['results'])
    output['status'] = results.get('status')
    if return_full_response is True:
        output['response'] = results
    
    return output


In [6]:
len(addresses)

1136

In [25]:
# Ensure, before we start, that the API key is ok/valid, and internet access is ok
# test_result = get_google_results("New York City, NY", API_KEY, RETURN_FULL_RESULTS)
# if (test_result['status'] != 'OK') or (test_result['formatted_address'] != 'London, UK'):
#     logger.warning("There was an error when testing the Google Geocoder.")
#     raise ConnectionError('Problem with test results from Google Geocode - check your API key and internet connection.')

# Create a list to hold results
results = []
# Go through each address in turn
for address in addresses:
    # While the address geocoding is not finished:
    geocoded = False
    while geocoded is not True:
        # Geocode the address with google
        try:
            geocode_result = get_google_results(address, API_KEY, return_full_response=RETURN_FULL_RESULTS)
        except Exception as e:
            logger.exception(e)
            logger.error("Major error with {}".format(address))
            logger.error("Skipping!")
            geocoded = True
            
        # If we're over the API limit, backoff for a while and try again later.
        if geocode_result['status'] == 'OVER_QUERY_LIMIT':
            logger.info("Hit Query Limit! Backing off for a bit.")
            time.sleep(BACKOFF_TIME * 60) # sleep for 30 minutes
            geocoded = False
        else:
            # If we're ok with API use, save the results
            # Note that the results might be empty / non-ok - log this
            if geocode_result['status'] != 'OK':
                logger.warning("Error geocoding {}: {}".format(address, geocode_result['status']))
            logger.debug("Geocoded: {}: {}".format(address, geocode_result['status']))
            results.append(geocode_result)           
            geocoded = True

    # Print status every 100 addresses
    if len(results) % 100 == 0:
    	logger.info("Completed {} of {} address".format(len(results), len(addresses)))
            
    # Every 500 addresses, save progress to file(in case of a failure so you have something!)
    if len(results) % 500 == 0:
        pd.DataFrame(results).to_csv("{}_bak".format(output_filename))
""
# All done
logger.info("Finished geocoding all addresses")
# Write the full results to csv using the pandas library.
pd.DataFrame(results).to_csv(output_filename, encoding='utf8')

Geocoded: 132 West 138th Street, Manhattan, NY: OK
Geocoded: 132 West 138th Street, Manhattan, NY: OK
Geocoded: 132 West 138th Street, Manhattan, NY: OK
Geocoded: 132 West 138th Street, Manhattan, NY: OK
Geocoded: 2309 Frederick Douglass Blvd, Manhattan, NY: OK
Geocoded: 2309 Frederick Douglass Blvd, Manhattan, NY: OK
Geocoded: 2309 Frederick Douglass Blvd, Manhattan, NY: OK
Geocoded: 2309 Frederick Douglass Blvd, Manhattan, NY: OK
Geocoded: 140 Flatbush Avenue, Brooklyn, NY: OK
Geocoded: 140 Flatbush Avenue, Brooklyn, NY: OK
Geocoded: 140 Flatbush Avenue, Brooklyn, NY: OK
Geocoded: 140 Flatbush Avenue, Brooklyn, NY: OK
Geocoded: 57 Bay Street, Staten Island, NY: OK
Geocoded: 57 Bay Street, Staten Island, NY: OK
Geocoded: 57 Bay Street, Staten Island, NY: OK
Geocoded: 57 Bay Street, Staten Island, NY: OK
Geocoded: Bay 32nd St & Falcon Ave, Far Rockaway, Queens, NY: OK
Geocoded: Bay 32nd St & Falcon Ave, Far Rockaway, Queens, NY: OK
Geocoded: Bay 32nd St & Falcon Ave, Far Rockaway, Quee

Geocoded: 4802 10th Avenue, Brooklyn, NY: OK
Geocoded: 4006 3rd Avenue, Bronx, NY: OK
Geocoded: 4006 3rd Avenue, Bronx, NY: OK
Geocoded: 4006 3rd Avenue, Bronx, NY: OK
Geocoded: 4006 3rd Avenue, Bronx, NY: OK
Geocoded: 140 58th Street, Brooklyn, NY: OK
Geocoded: 140 58th Street, Brooklyn, NY: OK
Geocoded: 140 58th Street, Brooklyn, NY: OK
Geocoded: 140 58th Street, Brooklyn, NY: OK
Geocoded: 36-02 126th St, Cars enter through gate 11, Queens, NY: OK
Geocoded: 36-02 126th St, Cars enter through gate 11, Queens, NY: OK
Geocoded: 36-02 126th St, Cars enter through gate 11, Queens, NY: OK
Geocoded: 36-02 126th St, Cars enter through gate 11, Queens, NY: OK
Geocoded: 200-16 Hollis Avenue, Queens, NY: OK
Geocoded: 200-16 Hollis Avenue, Queens, NY: OK
Geocoded: 200-16 Hollis Avenue, Queens, NY: OK
Geocoded: 200-16 Hollis Avenue, Queens, NY: OK
Geocoded: 919 2nd Avenue, Manhattan, NY: OK
Geocoded: 919 2nd Avenue, Manhattan, NY: OK
Geocoded: 919 2nd Avenue, Manhattan, NY: OK
Geocoded: 919 2nd A

Geocoded: 212-32 48th Ave, Queens, NY: OK
Geocoded: 212-32 48th Ave, Queens, NY: OK
Geocoded: 327 Beach 19th Street, Queens, NY: OK
Geocoded: 327 Beach 19th Street, Queens, NY: OK
Geocoded: 327 Beach 19th Street, Queens, NY: OK
Geocoded: 327 Beach 19th Street, Queens, NY: OK
Geocoded: 97 Columbia Heights, Brooklyn, NY: OK
Geocoded: 97 Columbia Heights, Brooklyn, NY: OK
Geocoded: 97 Columbia Heights, Brooklyn, NY: OK
Geocoded: 97 Columbia Heights, Brooklyn, NY: OK
Geocoded: 194 Franklin Ave, Brooklyn, NY: OK
Geocoded: 194 Franklin Ave, Brooklyn, NY: OK
Geocoded: 194 Franklin Ave, Brooklyn, NY: OK
Geocoded: 194 Franklin Ave, Brooklyn, NY: OK
Geocoded: Baychester Ave & Crawford Ave, Bronx, NY 10466, Bronx, NY: OK
Geocoded: Baychester Ave & Crawford Ave, Bronx, NY 10466, Bronx, NY: OK
Geocoded: Baychester Ave & Crawford Ave, Bronx, NY 10466, Bronx, NY: OK
Geocoded: Baychester Ave & Crawford Ave, Bronx, NY 10466, Bronx, NY: OK
Geocoded: Robard Lane & Hollis Ave, Queens, NY: OK
Geocoded: Rob

Geocoded: 2042 86 Street, -, Brooklyn, NY: OK
Geocoded: 7402 Grand Avenue, A, Queens, NY: OK
Geocoded: 7402 Grand Avenue, A, Queens, NY: OK
Geocoded: 7402 Grand Avenue, A, Queens, NY: OK
Geocoded: 7402 Grand Avenue, A, Queens, NY: OK
Geocoded: 6009 8th Avenue, Brooklyn, NY: OK
Geocoded: 6009 8th Avenue, Brooklyn, NY: OK
Geocoded: 6009 8th Avenue, Brooklyn, NY: OK
Geocoded: 6009 8th Avenue, Brooklyn, NY: OK
Geocoded: 818 57th Street, Brooklyn, NY: OK
Geocoded: 818 57th Street, Brooklyn, NY: OK
Geocoded: 818 57th Street, Brooklyn, NY: OK
Geocoded: 818 57th Street, Brooklyn, NY: OK
Geocoded: 807 Kings Highway, Brooklyn, NY: OK
Geocoded: 807 Kings Highway, Brooklyn, NY: OK
Geocoded: 807 Kings Highway, Brooklyn, NY: OK
Geocoded: 807 Kings Highway, Brooklyn, NY: OK
Geocoded: 62-04B Woodhaven Boulevard, Queens, NY: OK
Geocoded: 62-04B Woodhaven Boulevard, Queens, NY: OK
Geocoded: 62-04B Woodhaven Boulevard, Queens, NY: OK
Geocoded: 62-04B Woodhaven Boulevard, Queens, NY: OK
Geocoded: 3475 Thi

Geocoded: 75-51 Metropolitan Avenue, Queens, NY: OK
Geocoded: 75-51 Metropolitan Avenue, Queens, NY: OK
Geocoded: 75-51 Metropolitan Avenue, Queens, NY: OK
Geocoded: 394 Avenue P, Brooklyn, NY: OK
Geocoded: 394 Avenue P, Brooklyn, NY: OK
Geocoded: 394 Avenue P, Brooklyn, NY: OK
Geocoded: 394 Avenue P, Brooklyn, NY: OK
Geocoded: 898 East Tremont Avenue, Bronx, NY: OK
Geocoded: 898 East Tremont Avenue, Bronx, NY: OK
Geocoded: 898 East Tremont Avenue, Bronx, NY: OK
Geocoded: 898 East Tremont Avenue, Bronx, NY: OK
Geocoded: 2975 Richmond Ave, Staten Island, NY: OK
Geocoded: 2975 Richmond Ave, Staten Island, NY: OK
Geocoded: 2975 Richmond Ave, Staten Island, NY: OK
Geocoded: 2975 Richmond Ave, Staten Island, NY: OK
Geocoded: 32-50 Vernon Boulevard, Queens, NY: OK
Geocoded: 32-50 Vernon Boulevard, Queens, NY: OK
Geocoded: 32-50 Vernon Boulevard, Queens, NY: OK
Geocoded: 32-50 Vernon Boulevard, Queens, NY: OK
Geocoded: 517 East 117th Street, Manhattan, NY: OK
Geocoded: 517 East 117th Street, 

...

Geocoded: 136-20 38th Street, 5J, Queens, NY: OK
Geocoded: 136-20 38th Street, 5J, Queens, NY: OK
Geocoded: 136-20 38th Street, 5J, Queens, NY: OK
Geocoded: 136-20 38th Street, 5J, Queens, NY: OK
Geocoded: 38-21 Main Street, Suite 1A, Queens, NY: OK
Geocoded: 38-21 Main Street, Suite 1A, Queens, NY: OK
Geocoded: 38-21 Main Street, Suite 1A, Queens, NY: OK
Geocoded: 38-21 Main Street, Suite 1A, Queens, NY: OK
Geocoded: 1240 Walton Avenue, Suite1, Bronx, NY: OK
Geocoded: 1240 Walton Avenue, Suite1, Bronx, NY: OK
Geocoded: 1240 Walton Avenue, Suite1, Bronx, NY: OK
Geocoded: 1240 Walton Avenue, Suite1, Bronx, NY: OK
Geocoded: 1727 Amsterdam Avenue, 1st Floor, Manhattan, NY: OK
Geocoded: 1727 Amsterdam Avenue, 1st Floor, Manhattan, NY: OK
Geocoded: 1727 Amsterdam Avenue, 1st Floor, Manhattan, NY: OK
Geocoded: 1727 Amsterdam Avenue, 1st Floor, Manhattan, NY: OK
Geocoded: 8778 Bay Parkway, 2nd Floor, Brooklyn, NY: OK
Geocoded: 8778 Bay Parkway, 2nd Floor, Brooklyn, NY: OK
Geocoded: 8778 Bay P

Geocoded: 2324 Forest Avenue, Staten Island, NY: OK
Geocoded: 715 9th Avenue, Manhattan, NY: OK
Geocoded: 715 9th Avenue, Manhattan, NY: OK
Geocoded: 715 9th Avenue, Manhattan, NY: OK
Geocoded: 715 9th Avenue, Manhattan, NY: OK
Geocoded: 225 West 35th Street, Manhattan, NY: OK
Geocoded: 225 West 35th Street, Manhattan, NY: OK
Geocoded: 225 West 35th Street, Manhattan, NY: OK
Geocoded: 225 West 35th Street, Manhattan, NY: OK
Geocoded: 2431 86th Street, Brooklyn, NY: OK
Geocoded: 2431 86th Street, Brooklyn, NY: OK
Geocoded: 2431 86th Street, Brooklyn, NY: OK
Geocoded: 2431 86th Street, Brooklyn, NY: OK
Geocoded: 2083 East 65th Street, Brooklyn, NY: OK
Geocoded: 2083 East 65th Street, Brooklyn, NY: OK
Geocoded: 2083 East 65th Street, Brooklyn, NY: OK
Geocoded: 2083 East 65th Street, Brooklyn, NY: OK
Geocoded: 71-28 Cooper Avenue, Queens, NY: OK
Geocoded: 71-28 Cooper Avenue, Queens, NY: OK
Geocoded: 71-28 Cooper Avenue, Queens, NY: OK
Geocoded: 71-28 Cooper Avenue, Queens, NY: OK
Geocoded

Geocoded: 1711 Ralph Avenue, Brooklyn, NY: OK
Geocoded: 1711 Ralph Avenue, Brooklyn, NY: OK
Geocoded: 1711 Ralph Avenue, Brooklyn, NY: OK
Geocoded: 31-47 137th Street, Suite CF, Queens, NY: OK
Geocoded: 31-47 137th Street, Suite CF, Queens, NY: OK
Geocoded: 31-47 137th Street, Suite CF, Queens, NY: OK
Geocoded: 31-47 137th Street, Suite CF, Queens, NY: OK
Geocoded: 45-02 82nd Street, Queens, NY: OK
Geocoded: 45-02 82nd Street, Queens, NY: OK
Geocoded: 45-02 82nd Street, Queens, NY: OK
Geocoded: 45-02 82nd Street, Queens, NY: OK
Geocoded: 87-35 Britton Ave, Queens, NY: OK
Geocoded: 87-35 Britton Ave, Queens, NY: OK
Geocoded: 87-35 Britton Ave, Queens, NY: OK
Geocoded: 87-35 Britton Ave, Queens, NY: OK
Geocoded: 36-36 Main Street, Queens, NY: OK
Geocoded: 36-36 Main Street, Queens, NY: OK
Geocoded: 36-36 Main Street, Queens, NY: OK
Geocoded: 36-36 Main Street, Queens, NY: OK
Geocoded: 754 East 151st Street, 2nd Floor, Bronx, NY: OK
Geocoded: 754 East 151st Street, 2nd Floor, Bronx, NY: O

Geocoded: 1110 Pennsylvania Avenue, Brooklyn, NY: OK
Geocoded: 501 5th Avenue, Brooklyn, NY: OK
Geocoded: 501 5th Avenue, Brooklyn, NY: OK
Geocoded: 501 5th Avenue, Brooklyn, NY: OK
Geocoded: 501 5th Avenue, Brooklyn, NY: OK
Geocoded: 372 East 204th Street, Bronx, NY: OK
Geocoded: 372 East 204th Street, Bronx, NY: OK
Geocoded: 372 East 204th Street, Bronx, NY: OK
Geocoded: 372 East 204th Street, Bronx, NY: OK
Geocoded: 932 East 174th Street, Bronx, NY: OK
Geocoded: 932 East 174th Street, Bronx, NY: OK
Geocoded: 932 East 174th Street, Bronx, NY: OK
Geocoded: 932 East 174th Street, Bronx, NY: OK
Geocoded: 37-15 23rd Avenue, Queens, NY: OK
Geocoded: 37-15 23rd Avenue, Queens, NY: OK
Geocoded: 37-15 23rd Avenue, Queens, NY: OK
Geocoded: 37-15 23rd Avenue, Queens, NY: OK
Geocoded: 2063A Bartow Avenue, Bronx, NY: OK
Geocoded: 2063A Bartow Avenue, Bronx, NY: OK
Geocoded: 2063A Bartow Avenue, Bronx, NY: OK
Geocoded: 2063A Bartow Avenue, Bronx, NY: OK
Completed 1100 of 1136 address
Completed 11

Finished geocoding all addresses
Finished geocoding all addresses
Finished geocoding all addresses


In [27]:
latlng = pd.DataFrame(results)
latlng = latlng[['formatted_address', 'latitude', 'longitude',]]
latlng.head(2)

,formatted_address,latitude,longitude
0,"132 W 138th St, New York, NY 10030, USA",40.816682,-73.941556
1,"2309 Frederick Douglass Blvd, New York, NY 100...",40.809686,-73.951782


In [30]:
df.head(2)

,Name,Address,Notes,Phone_Number,Station_Type,Vaccine_Type,Notes
0,Abyssinian Baptist Church- Pop Up,"132 West 138th Street, Manhattan","$100 incentive available , Walk-up vaccination...",(877) 829-4692,Pop Up - Van,Vaccines offered:Pfizer (12+)Johnson & Johnson...,"$100 incentive available , Walk-up vaccination..."
1,AMC Magic Johnson Harlem- Pop Up,"2309 Frederick Douglass Blvd, Manhattan","$100 incentive available , Walk-up vaccination...",(877) 829-4692,Pop Up - Bus,Vaccine offered:Pfizer (5-11),"$100 incentive available , Walk-up vaccination..."


In [35]:
vacStn = pd.concat([df, latlng], axis=1)
vacStn.head(2)

,Name,Address,Notes,Phone_Number,Station_Type,Vaccine_Type,Notes,formatted_address,latitude,longitude
0,Abyssinian Baptist Church- Pop Up,"132 West 138th Street, Manhattan","$100 incentive available , Walk-up vaccination...",(877) 829-4692,Pop Up - Van,Vaccines offered:Pfizer (12+)Johnson & Johnson...,"$100 incentive available , Walk-up vaccination...","132 W 138th St, New York, NY 10030, USA",40.816682,-73.941556
1,AMC Magic Johnson Harlem- Pop Up,"2309 Frederick Douglass Blvd, Manhattan","$100 incentive available , Walk-up vaccination...",(877) 829-4692,Pop Up - Bus,Vaccine offered:Pfizer (5-11),"$100 incentive available , Walk-up vaccination...","2309 Frederick Douglass Blvd, New York, NY 100...",40.809686,-73.951782


In [37]:
vacStn.to_csv('Vac_station_final.csv')